In [4]:
# Import Libraries
import plotly.express as px
import chart_studio.plotly as py
import chart_studio

from datetime import timedelta, datetime
import pandas as pd
import numpy as np
import pickle
import sys, os

In [6]:
absolutepath = os.path.abspath('')
fileDirectory = os.path.dirname(absolutepath)

#Path of parent directory (moves outside of repository)
parentDirectory = os.path.dirname(fileDirectory)

In [8]:
fileDirectory

'/Users/chris/Documents/GitHub/financial_forecasting_analysis'

In [5]:
## Connect with chartstudio
username = 'augurychris' # Add your username
api_key = 'Gja5FFkgBceJpvJSXOA6' # Add your api key 

chart_studio.tools.set_credentials_file(username = username, api_key = api_key)

In [122]:
df = pd.DataFrame()
tick = 'WAT'
files = {
    'LinReg':f'linreg_preds/{tick}_linreg_preds.csv',
    'Arima':f'arima_preds/{tick}_arima_preds.csv',
    'RandFor':f'rfreg_preds/{tick}_rfreg_preds.csv',
    #'XGBoost':f'xgboost_preds/{tick}_1dayhigh_xgbpreds.csv',
    'LSTM_Spline':f'LSTM_preds/{tick}_backtesting_LSTM_SPLINE_predictions.csv',
    }

In [123]:
for k,v in files.items():
    tmp_df = pd.read_csv(fileDirectory + '/data/ticker_predictions/' + v)
    tmp_df['model'] = k 
    tmp_df['timestamp'] = pd.to_datetime(tmp_df['timestamp']) + timedelta(days=1)
    df = pd.concat([df,tmp_df])
    del df['Unnamed: 0']

In [124]:
xg = pd.read_csv(fileDirectory + '/data/ticker_predictions/' + f'xgboost_preds/{tick}_1dayhigh_xgbpreds.csv')
xg = xg[xg.timestamp >= '2019-09-16'][['timestamp','holdout_preds']]
xg['timestamp'] = pd.to_datetime(xg['timestamp'])
xg['timestamp'] = xg['timestamp'] + timedelta(days=1)
xg.rename(columns={'holdout_preds':'prediction'}, inplace=True)
xg['model'] = 'XGBoost'

In [125]:
# add in true data
ticker_df = pd.read_pickle(fileDirectory + '/data/raw_data_2021-11-12_12.31.45.pkl')
coi = ['timestamp','High']
ticker_df['timestamp'] = pd.to_datetime(ticker_df['reportperiod'])
ticker_df = ticker_df[(ticker_df.ticker == tick) & (ticker_df.timestamp >= '2019-09-16')][coi]
ticker_df['model'] = 'Actual'
ticker_df.rename(columns={'High':'prediction'}, inplace=True)

In [ ]:
ticker_df

In [126]:
combined = pd.concat([df,xg,ticker_df],ignore_index=True)

In [127]:
combined = combined[(combined.timestamp <= '2021-11-10')]

In [131]:
# example
stock_chart = px.line(combined, x='timestamp', y='prediction', color='model',
             title='Test Set: Forecasting Waters Corporation (WAT) Next Day High',labels={'prediction':"Stock Price [High]",'timestamp':'Date'})

#pop.update_traces(textposition='inside', textinfo='percent+label')

stock_chart.update_traces(mode="markers+lines", hovertemplate=None)
stock_chart.update_layout(hovermode="x unified")
stock_chart.show()

In [132]:
# export
py.plot(stock_chart, filename = 'Forecasting Waters Corporation (WAT) Next Day High', auto_open=True)# Output
#https://chart-studio.plotly.com/~augurychris/1/#/

'https://plotly.com/~augurychris/1/'